Created on Tue Apr 21 11:10:11 2020

@author: Deeps

In [1]:
!pip install torch
!pip install torchsummary
! pip install torchvision

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar

import torch
from torch.utils import data
from torchsummary import summary
from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split as tts
import torch.nn.functional as F
from torchvision import datasets,models, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split
from data_gen import Dataset
from augment import Augmentations as aug

## Load the dataset and train, val, test splits

In [8]:
num_epochs = 70
num_classes = 2
batch_size = 8
learning_rate = 0.001
datacsv = pd.read_csv("data.csv")

cv_results = []

### ------------Data Augmentation--------

In [5]:
data_base_aug = dict(datacsv)
data_base_aug["image"] = list(data_base_aug["image"])
data_base_aug["label"] = list(data_base_aug["label"])

rot = aug()
for k in range(len(data_base_aug["image"])):
    result = rot.rotate_append(data_base_aug["image"][k],data_base_aug["label"][k])
    datacsv = datacsv.append(result,ignore_index = True)

_____

# ----------Create dataset generator-------------

In [9]:
for cv in range(3):
    x_train,x_test =tts(datacsv["image"],test_size=1/6, shuffle=True)
    partition = {'train':list(x_train),'validation':list(x_test)}
    labels = {}
    
    for k in range(len(datacsv['image'])):
        labels['%s'%datacsv['image'][k]] = datacsv['label'][k]
        
    #####################################################################
    
    training_set = Dataset(partition["train"],labels)
    training_generator = data.DataLoader(training_set, batch_size=batch_size, shuffle=True)
    
    validation_set = Dataset(partition['validation'], labels)
    validation_generator = data.DataLoader(validation_set, batch_size=batch_size, shuffle=True)
    
    
    
    #######################################################
    class Network(nn.Module):
        def __init__(self):
            super().__init__()
            # TODO: Design your own network, define layers here.
            # Here We provide a sample of two-layer fully-connected network.
            # Your solution, however, should contain convolutional layers.
            # Refer to PyTorch documentations of torch.nn to pick your layers. (https://pytorch.org/docs/stable/nn.html)
            # Some common Choices are: Linear, Conv2d, ReLU, MaxPool2d, AvgPool2d, Dropout
            # If you have many layers, consider using nn.Sequential() to simplify your code
            # self.fc1 = nn.Linear(28*28, 8) # from 28x28 input image to hidden layer of size 256
            # self.fc2 = nn.Linear(8,10) # from hidden layer to 10 class scores
            self.conv1 = nn.Conv2d(3, 6, 5)
            self.conv2 = nn.Conv2d(6, 12, 5)
            self.conv3 = nn.Conv2d(12, 6, (5,4))
            self.conv4 = nn.Conv2d(6, 3, (7,4))
            self.pool = nn.MaxPool2d(2, 2)
            self.poolav = nn.AvgPool2d(2,2)
            self.poollast = nn.MaxPool2d(3, 2) 
            self.fc1 = nn.Linear(3 * 42 * 29, 258)
            self.fc2 = nn.Linear(258, num_classes)
            
    
        def forward(self,x):
            # TODO: Design your own network, implement forward pass here
            x = self.poolav(F.relu(self.conv1(x))) #3*748*512 -> 6*744*508 -> 6*372*254
            x = self.pool(F.relu(self.conv2(x))) #6*372*254 -> 12*368*250 -> 12*184*125
            x = self.poolav(F.relu(self.conv3(x))) #12*184*125 -> 6*180*122 -> 6*90*61
            x = self.pool(F.relu(self.conv4(x))) #6*90*61-> 3*84*58 -> 3*42*29
            x = x.view(-1, 3 * 42 * 29)
            x = F.dropout(F.relu(self.fc1(x)),0.2)
            out = F.softmax(self.fc2(x))
            #out = F.softmax(self.fc3(x))
            return out
    
    
    device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
    model = Network().to(device)
    summary(model,(3,748,512))
    criterion = nn.CrossEntropyLoss() # Specify the loss layer
    optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9) # Specify optimizer and assign trainable parameters to it, weight_decay is L2 regularization strength
    
    
    
    def train(model, training_generator, num_epoch = num_epochs): # Train the model
        print("Start training...")
        model.train() # Set the model to training mode
        for i in range(num_epoch):
            running_loss = []
            for batch, label in tqdm(training_generator):
                batch = batch.to(device)
                label = label.to(device)
                optimizer.zero_grad() # Clear gradients from the previous iteration
                pred = model(batch) # This will call Network.forward() that you implement
                loss = criterion(pred, label) # Calculate the loss
                running_loss.append(loss.item())
                loss.backward() # Backprop gradients to all tensors in the network
                optimizer.step() # Update trainable weights
            print("Epoch {} loss:{}".format(i+1,np.mean(running_loss))) # Print the average loss for this epoch
        print("Done!")
        try:
            torch.save(model, "Trained_model/teeth_model.pth")
            print("Model saved!")
        except:
            print("Could not save model")
    
    def evaluate(model, validation_generator): # Evaluate accuracy on validation / test set
        model.eval() # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.set_grad_enabled(True): # Do not calculate grident to speed up computation
            for batch, label in tqdm(validation_generator):
                batch = batch.to(device)
                label = label.to(device)
                pred = model(batch)
                correct += (torch.argmax(pred,dim=1)==label).sum().item()
                total+=batch_size
        acc = correct/total
        print("Evaluation accuracy: {}".format(acc))
        return acc
        
    train(model, training_generator, num_epochs)
    print("Evaluate on validation set...")
    result = evaluate(model, validation_generator)
    cv_results.append(result)


/home/moselim/.conda/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  8%|▊         | 1/13 [00:00<00:01,  7.28it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 744, 508]             456
         AvgPool2d-2          [-1, 6, 372, 254]               0
            Conv2d-3         [-1, 12, 368, 250]           1,812
         MaxPool2d-4         [-1, 12, 184, 125]               0
            Conv2d-5          [-1, 6, 180, 122]           1,446
         AvgPool2d-6            [-1, 6, 90, 61]               0
            Conv2d-7            [-1, 3, 84, 58]             507
         MaxPool2d-8            [-1, 3, 42, 29]               0
            Linear-9                  [-1, 258]         942,990
           Linear-10                    [-1, 2]             518
Total params: 947,729
Trainable params: 947,729
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 4.38
Forward/backward pass size (MB): 33.55
Params size (MB): 3.62
Estimated 

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch 1 loss:0.6794014664796683


  0%|          | 0/13 [00:00<?, ?it/s]

...

Epoch 69 loss:0.41907657109774077


 33%|███▎      | 1/3 [00:00<00:00,  8.25it/s]

Epoch 70 loss:0.3998765922509707
Done!
Model saved!
Evaluate on validation set...


100%|██████████| 3/3 [00:00<00:00,  9.22it/s]

Evaluation accuracy: 0.625
